# E213 Module 2 – Live Session: Data Acquisition & Financial Ratios (Alpha Vantage)

In this notebook, we will:
1. Load a secret API key from a `.env` file (so we don't hard-code secrets).
2. Call the **Alpha Vantage** API to get real company financials.
3. Convert JSON (raw API data) into Pandas DataFrames.
4. Clean numbers and align tables by date.
5. Compute key ratios: **Current Ratio**, **Debt-to-Equity**, **Gross/Operating/Net Margins**, **ROA**, **ROE**.

## 0) One-time Setup (do before running the rest)
1. Install packages (in a terminal):
   ```bash
   pip install pandas requests python-dotenv
   ```
2. Create a file named **`.env`** in the **same folder** as this notebook with this content:
   ```
   ALPHAVANTAGE_API_KEY=YOUR_KEY_HERE
   ```
3. Get a free key from Alpha Vantage: https://www.alphavantage.co/support/#api-key

**Why `.env`?** It keeps secrets out of your code and out of Canvas submissions.

## 1) Imports & Loading the API Key

In [ ]:
import os              # for reading environment variables (for example: API key)
import requests        # for web APIs (HTTP) calls
import pandas as pd    # data structure +
from dotenv import load_dotenv  # get environment variables from .env file

# .env file must be in the current (Jupyter-Notebook home) folder 
load_dotenv()

# Get API key 
API_KEY = os.getenv("ALPHAVANTAGE_API_KEY")

# optional: missing key error handling
if not API_KEY:
    raise SystemExit("ALPHAVANTAGE_API_KEY not found. Create a .env file with ALPHAVANTAGE_API_KEY=YOUR_KEY_HERE")

# AAPL, MSFT, AMZN, etc. - stocker ticker
SYMBOL = "AAPL"

# Base URL for Alpha Vantage.
BASE = "https://www.alphavantage.co/query"


## 2) A tiny helper function to call the API safely given the free API key limits
- Here is a function created for this class to make API calls.
- It adds the function name (like `INCOME_STATEMENT`) and our symbol.
- It also checks for common API issues:
  - **Rate limit** (too many requests/minute) → shows a helpful message.
  - **Error Message** (bad parameters) → shows a helpful message.

In [ ]:
def av_fetch(function, symbol):
    """
    Fetch data from Alpha Vantage.

    Arguments:
        function -- the API function name (e.g., "TIME_SERIES_DAILY")
        symbol -- the stock ticker symbol (e.g., "AAPL")

    Returns:
        A Python dictionary created from the JSON data returned by Alpha Vantage.
    """
    
    # Create a dictionary of parameters for the web request.
    # These key-value pairs will be sent to Alpha Vantage.
    params = {
        "function": function,
        "symbol": symbol,
        "apikey": API_KEY
    }

    # Make a web request to Alpha Vantage using the base URL and our parameters.
    response = requests.get(BASE, params=params, timeout=20)

    # If Alpha Vantage returns a non-OK (not 200) HTTP status code, raise an error.
    response.raise_for_status()

    # Convert the JSON response text into a Python dictionary or list.
    data = response.json()

    # If we hit the free-tier limit, Alpha Vantage includes a "Note" in the data.
    # Example: 
    # "Note": "Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day."
    if isinstance(data, dict) and "Note" in data: 
        raise RuntimeError("Rate limit hit. Wait ~60 seconds and run the cell again.")

    # If the function or symbol is wrong, Alpha Vantage includes an "Error Message".
    if isinstance(data, dict) and "Error Message" in data:
        raise RuntimeError(f"API error: {data['Error Message']}")

    # Return the Python dictionary to the caller.
    return data


## 3) Download **Income Statement** and **Balance Sheet**
**Annual Data**. Alpha Vantage returns JSON with keys like `annualReports` and `quarterlyReports`.

In [ ]:
# 3A) Income Statement
inc = av_fetch("INCOME_STATEMENT", SYMBOL)
                    # use dict.get(key, default) to retrieve data -- Alternatively inc["annualReports"] but not safe.
inc_annual = pd.DataFrame(inc.get("annualReports", []))
print("Income statement columns:")
print(list(inc_annual.columns))
inc_annual.head(3)

In [ ]:
# 3B) Balance Sheet
bal = av_fetch("BALANCE_SHEET", SYMBOL)
bal_annual = pd.DataFrame(bal.get("annualReports", []))
print("Balance sheet columns:")
print(list(bal_annual.columns))
bal_annual.head(3)

## 4) Select the columns we want
Only those needed for ratios.

- From the **income statement**:
  - `totalRevenue`, `grossProfit`, `operatingIncome`, `netIncome`
- From the **balance sheet**:
  - `totalAssets`, `totalLiabilities`, `totalShareholderEquity`, `totalCurrentAssets`, `totalCurrentLiabilities`

`fiscalDateEnding` (the period label) is also needed.

In [ ]:
keep_inc = [
    "fiscalDateEnding","totalRevenue","grossProfit","operatingIncome","netIncome"
]
keep_bal = [
    "fiscalDateEnding","totalAssets","totalLiabilities","totalShareholderEquity",
    "totalCurrentAssets","totalCurrentLiabilities"
]

inc_small = inc_annual[keep_inc].copy()
bal_small = bal_annual[keep_bal].copy()

inc_small.head(3)

In [ ]:
bal_small.head(3)

## 5) Convert numbers from **text** to **numeric**
APIs often return numbers as **strings**. We will convert them so we can do math.

We use `pd.to_numeric(..., errors='coerce')` which turns bad values into `NaN` instead of crashing.

In [ ]:
inc_num = inc_small.copy()
bal_num = bal_small.copy()

# Convert all columns except 'fiscalDateEnding' to numeric
for col in inc_num.columns:
    if col != "fiscalDateEnding":
        inc_num[col] = pd.to_numeric(inc_num[col], errors="coerce")

for col in bal_num.columns:
    if col != "fiscalDateEnding":
        bal_num[col] = pd.to_numeric(bal_num[col], errors="coerce")


In [ ]:
print(inc_num.dtypes.head()) 
inc_num.head(3)

In [ ]:
print(bal_num.dtypes.head())
bal_num.head(3)

## 6) Align the two tables by **fiscalDateEnding** and sort by date
We set the date as the **index** so we can join the tables on the same period.

In [ ]:
merged = (
    inc_num.set_index("fiscalDateEnding")[["totalRevenue","grossProfit","operatingIncome","netIncome"]]
    .join(
        bal_num.set_index("fiscalDateEnding")[["totalAssets","totalLiabilities","totalShareholderEquity","totalCurrentAssets","totalCurrentLiabilities"]],
        how="inner"
    )
    .sort_index()
)
merged.tail(6)

## 7) Compute the financial ratios
Formulas:
- **Current Ratio** = Total Current Assets / Total Current Liabilities
- **Debt-to-Equity** = Total Liabilities / Total Shareholder Equity
- **Gross Margin** = Gross Profit / Revenue
- **Operating Margin** = Operating Income / Revenue
- **Net Margin** = Net Income / Revenue
- **ROA** ≈ Net Income / Average Assets
- **ROE** ≈ Net Income / Average Equity

We approximate **Average Assets** (and **Average Equity**) by averaging this year with the previous year.

In [ ]:
ratios = merged.copy()

# Liquidity & Leverage
ratios["currentRatio"]   = ratios["totalCurrentAssets"] / ratios["totalCurrentLiabilities"]
ratios["debtToEquity"]   = 0  # Plaholder

# Profitability margins
ratios["grossMargin"]     = 0  # Plaholder
ratios["operatingMargin"] = 0  # Plaholder
ratios["netMargin"]       = 0  # Plaholder

# ROA & ROE (using period-average assets/equity)

ratios["ROA"] = 0  # Plaholder
ratios["ROE"] = 0  # Plaholder

ratios.tail(6)

## 8) Print a clean summary for the **latest** period
We will format the ratios so they are easy to read.

In [ ]:
latest = ratios.tail(1)    # last row of the ratios DataFrame
period = latest.index[-1]  # index value of that last row (the most recent date)

def pct(x):
    return "N/A" if pd.isna(x) else f"{x:.2%}"

def num(x):
    return "N/A" if pd.isna(x) else f"{x:.2f}"

print("=== Financial Ratio Summary (", SYMBOL, ") ===", sep="")
print("Period:", period)
print("Current Ratio:         ", num(latest["currentRatio"].iloc[0]))
print("Debt-to-Equity:        ", num(latest["debtToEquity"].iloc[0]))
print("Gross Margin:          ", pct(latest["grossMargin"].iloc[0]))
print("Operating Margin:      ", pct(latest["operatingMargin"].iloc[0]))
print("Net Margin:            ", pct(latest["netMargin"].iloc[0]))
print("ROA:                   ", pct(latest["ROA"].iloc[0]))
print("ROE:                   ", pct(latest["ROE"].iloc[0]))


## 9) Report
Your clear interpretation and business context